#  Object Creation and Initialization Process in Python

##  Table of Contents 
* [The `__new__` method](#new-method)
* [The `__init__` method](#init-method)
* [The `__call__` method](#call-method)
* [Conclusion](#conclusion)

---

With a basic understanding of the Metaclass and objects in Python, let's now understand the object creation and initialization process in Python. Consider the `Human` class, as defined below:

In [1]:
class Human:
    def __init__(self, first_name, last_name):
        self.first_name = first_name
        self.last_name = last_name

In [2]:
human_obj = Human("Kian", "Pirfalak")

In [3]:
isinstance(human_obj, Human)

True

In [4]:
isinstance(human_obj, object)

True

The output of the above code shows that `human_obj` is an instance of class `Human` with the `first_name` as Kian and the `last_name` as Pirfalak. If we look at the above code closely, it's natural to have some questions:

1. Per the definition of the `Human` class, we don't return anything from the `__init__` method; how does calling the `Human` class return the `human_obj`?
2. We know that the `__init__` method is used for initializing the object, but how does the `__init__` method get self?

In this section, we will discuss each of these questions in detail and answer them.

**Object creation in Python is a two-step process**. In the first step, Python creates the object, and in the second step, it initializes the object. Most of the time, we are only interested in the second step (i.e., the initialization step). Python uses the `__new__` method in the first step (i.e., object creation) and uses the `__init__` method in the second step (i.e., initialization).

If the class does not define these methods, they are inherited from the object base class. As the `Human` class does not define the `__new__` method, during the object instantiation process, the `__new__` method of the object's class is called, while for initialization, the `__init__` method of the Human class is called. Next, we'll cover each of these methods in detail.

<a class="anchor" id="new-method"></a>
## The `__new__` method

The __new__ method is the first step in the object instantiation process. It is a static method (we will cover static methods in detail later, but you can think of it as a method that doesn't receive `self` in arguments) on the object class and accepts `cls` or the class reference as the first parameter. The remaining arguments (Kian and Pirfalak) are passed while calling the class - `Human('Kian', 'Pirfalak')`. The `__new__` method creates an instance of type `cls` (i.e., it allocates memory for the object by invoking the superclass' i.e. object class' `__new__` method using `super().__new__(cls))`. It then returns the instance of type `cls`.

Usually, it does not do any initialization, as that is the job of the `__init_`_ method. However, when you override the `__new__` method, you can also use it to initialize the object or modify it as required before returning it.

`__new__` method signature:

```python
# cls - is the mandatory argument. Object returned by the __new__ method is of type cls
@staticmethod
def __new__(cls[,...]):
    pass
```

We can modify the object creation process by overriding the `__new__` method of the object class. Consider the example below:

In [5]:
class Human:
    def __new__(cls, first_name=None):
        # cls = Human. cls is the class using which the object will be created.
        # Created object will be of type cls.
        # We must call the object class' __new__ to allocate memory
        obj = super().__new__(cls) # This is equivalent to object.__new__(cls)

        # Modify the object created
        if first_name:
            obj.name = first_name
        else:
            obj.name = "Kian"

        print(type(obj)) # Prints: <__main__.Human object at 0x103665668>
        # likely with a different memory location on your machine

        # return the object
        return obj

In [6]:
# Create an object
# __init__ method of `object` class will be called.
k = Human()

k.name

<class '__main__.Human'>


'Kian'

In [7]:
m = Human("Mahsa")
m.name

<class '__main__.Human'>


'Mahsa'

In the above example, we have overridden the `__new__` method of the object class. It accepts the first arguments as `cls` - a class reference to the `Human` class.

> If you know `staticmethods` already, note that the `__new__` method is a special case in Python. Although it's a static method of the object class, on overriding it, we do not have to decorate it with the `staticmethod` decorator

Inside the `__new__` method of the `Human` class, we are first calling the `__new__` method of the object class using `super().__new__(cls)`. The object class `__new__` method creates and returns the instance of the class, which is passed as an argument to the `__new__` method. Here, as we are passing `cls` (i.e., the `Human` class reference); the object's `__new__` method will return an instance of type `Human`.

> We must call the object class `__new__` method inside the overridden `__new__` method to create the object and allocate memory to the object.

> The `__new__` method of the `Human` class modifies the `obj` returned from the `__new__` method of the object class and adds the `name` property to it. Thus, all objects created using the `Human` class will have a name property. Voila! We have modified the object instantiation process of the `Human` class.

Let's consider another example. In this example, we are creating a new class called `Animal` and overriding the `__new__` method. Here, when we are calling the `__new__` method of the object class from the `__new__` method of the `Animal` class, instead of passing the `Animal` class reference as an argument to the `__new__` method of the object class, we are passing the `Human` class reference. Hence, the object returned from the `__new__` method of the object class will be of type `Human` and not `Animal`. As a result, the object returned from calling the `Animal` class (i.e., `Animal()`) will be of type `Human`.

In [8]:
class Animal:
    def __new__(cls):
        # Passing Human class reference instead of Animal class reference
        obj = super().__new__(Human) # This is equivalent to object.__new__(Human)

        print(f"Type of obj: {type(obj)}") # Prints: Type of obj: <class '__main__.Human'>

        # return the object
        return obj

In [9]:
cat = Animal()

Type of obj: <class '__main__.Human'>


In [10]:
type(cat)

__main__.Human

<a class="anchor" id="init-method"></a>
## The `__init__` method

The `__init__` method is the second step of the object instantiation process in Python. It takes the first argument as an object or instance returned from the `__new__` method. The remaining arguments are the arguments passed while calling the class (`Human("Kian", "Pirfalak")`). These arguments are used for initializing the object. The `__init__` method must not return anything. If you try to return anything using the `__init__` method, it will raise an exception, as shown below:

In [11]:
class Human:
    def __init__(self, first_name):
        self.first_name = first_name
        return self

In [12]:
human_obj = Human('Kian')

TypeError: __init__() should return None, not 'Human'

Consider a simple example to understand both the `__new__` and `__init__` method.

In [ ]:
class Human:
    def __new__(cls, *args, **kwargs):
        # Here, the __new__ method of the object class must be called to create
        # and allocate the memory to the object
        print("Inside new method")
        print(f"args arguments {args}")
        print(f"kwargs arguments {kwargs}")

        # The code below calls the __new__ method of the object's class.
        # Object class' __new__ method allocates a memory
        # for the instance and returns that instance
        human_obj = super(Human, cls).__new__(cls)

        print(f"human_obj instance - {human_obj}")
        return human_obj

    # As we have overridden the __init__ method, the __init__ method of the object class will not be called
    def __init__(self, first_name, last_name):
        print("Inside __init__ method")
        # self = human_obj returned from the __new__ method

        self.first_name = first_name
        self.last_name = last_name

        print(f"human_obj instance inside __init__ {self}: {self.first_name}, {self.last_name}")

In [ ]:
human_obj = Human("Kian", "Pirfalak")

In the above code, we have overridden both the `__new__` and `__init__` method of the object's class. `__new__` creates the object (`human_obj`) of type `Human` class and returns it. Once the `__new__` method is complete, Python calls the `__init__` method with the `human_obj` object as the first argument. The `__init__` method initializes the `human_obj` with `first_name` as Kian and `last_name` as Pirfalak. As object creation is the first step, and initialization is the second step, the `__new__` method will always be called before the `__init__` method

Both `__init__` and `__new__` are called magic methods in Python. Magic methods have names that begin and end with `__` (double underscores or "dunder"). Magic methods are called implicitly by the Python; you do not have to call them explicitly. For example, both the `__new__` and `__init__` method are called implicitly by Python. Let's cover one more magic method, `__call__`.

<a class="anchor" id="call-method"></a>
## The `__call__` method

The `__call__` method is a magic method in Python that is used to make the objects callable. Callable objects are objects that can be called. For example, functions are callable objects, as they can be called using the round parenthesis.

Consider an example to better understand callable objects:

In [ ]:
def print_function():
    print("I am a callable object")

# print_function is callable as it can be called using round parentheses
print_function()

Let's try to call an integer object. As integer objects are not callable, calling them will raise an exception.

In [ ]:
a = 10

In [ ]:
a()

The `callable` function is used to determine whether an object is callable. The `callable` function takes the object reference as an argument and returns `True` if the object appears to be callable or `False` if the object is not callable. If the callable function returns `True`, the object might not be callable; however, if it returns `False`, then the object is certainly not callable.

In [ ]:
# Functions are callable
callable(print_function)

In [ ]:
# Interger object is not callable
callable(a)

Let's determine whether the classes in Python are callable. Here, we will determine whether the `Human` class defined earlier is callable.

In [ ]:
callable(Human)

Yes, classes in Python are callable, and they should be! Don't you think so? When we call the class, it returns the instance of that class. Let's find out whether the objects created from the class are callable.

In [ ]:
human_obj = Human("Virat", "Kohli")

In [ ]:
# Let's try calling the human_obj
callable(human_obj)

So, `human_obj` is not callable though the class of `human_obj` (i.e., the `Human` class is callable)

To make any object in Python callable, Python provides the `__call__` method that needs to be implemented by the object's class. For example, to make `human_obj` object callable, the `Human` class has to implement the `__call__` method. Once the `Human` class implements the `__call__` method, all the objects of the `Human` class can be invoked like functions (i.e., using round parentheses)

In [ ]:
class Human:
    def __init__(self, first_name, last_name):
        print("I am inside __init__ method")
        self.first_name = first_name
        self.last_name = last_name

    def __call__(cls):
        print("I am inside __call__ method")

In [ ]:
human_obj = Human("Kian", "Pirfalak")

In [ ]:
human_obj()

In [ ]:
human_obj.__call__()

In [ ]:
callable(human_obj)

The above code output shows that after implementing the `__call__` method on the `Human` class, `human_obj` becomes a callable object. We can call the `human_obj` using round parentheses (i.e., `human_obj()`). When we use `human_obj()`, in the background, Python calls the `__call__` method of the `Human` class. So, instead of calling `human_obj` as `human_obj()`, we can directly invoke the `__call__` method on `human_obj` (i.e., `human_obj.__call__()`). Both `human_obj()` and `human_obj.__call__()` are equivalent, and they are the same thing.

> For all objects that are callable, their classes must implement the `__call__` method.

We know that functions are a callable object, so its class (i.e., function) must implement the `__call__` method. Let's invoke the `__call__` method on the `print_function` defined earlier.

In [ ]:
def print_function():
    print("I am a callable object")

In [ ]:
print_function.__call__()

> In Python, class is also a callable object; therefore, it is a class's class (metaclass) (i.e., the `type` class must have a call method defined on it). Hence, when we call `Human()`, in the background, Python calls the call method of the `type` class.

Roughly, the `__call__` method on the types class looks something like shown below. This is just for explanation purposes; we will cover the actual definition of the `__call__` method later in the tutorial.

```python
class type:
    def __call__():
        # Called when class is called i.e. Human()
        print("type's call method")
```

We know that when we call the class (i.e., `Human("Kian", "Pirfalak")`), the `__call__` method of the `type` class is called. However, what is the definition of the types class `__call__` method? As we are talking about CPython, the type class `__call__` method definition is defined in C language. If we convert it into Python and simplify it, it will look somewhat like this:

```python
# type's __call__ method which gets called when Human class is called i.e. Human()
def __call__(cls, *args, **kwargs):
    # cls = Human class
    # args = ["Virat", "Kohli"]
    # Calling __new__ method of the Human class, as __new__ method is not defined
    # on Human, __new__ method of the object class is called
    human_obj = cls.__new__(*args, **kwargs)

    # After __new__ method returns the object, __init__ method will only be called if
    # 1. human_obj is not None
    # 2. human_obj is an instance of class Human
    # 3. __init__ method is defined on the Human class
    if human_obj is not None and isinstance(human_obj, cls) and hasattr(human_obj, '__init__'):
        # As __init__ is called on human_obj, self will be equal to human_obj in __init__ method
        human_obj.init(*args, **kwargs)

    return human_obj
```

Let's understand the above code; when we do Human("Kian", "Pirfalak"), in the background, Python will call the type class `__call__` method, which is defined like the above code snippet. As shown above, the type class `__call__` method accepts `Human` class as the first argument (`cls` is `Human` class), and the remaining arguments are passed while calling the `Human` class. The type class `__call__` method will first call the `__new__` method defined on the `Human` class, if any; otherwise, the `__new__` method of the `Human` class parent class (i.e. the object's `__new__` method) is called. The `__new__` method will return the `human_obj`. Now, the `__call__` method of the type class will call the `__init__` method defined on the `Human` class with `human_obj` as the first argument. `__init__` will initialize the `human_obj` with the passed arguments, and finally, the `__call__` method will return the `human_obj`.

So, following steps are followed while creating and initializing an object in Python:

1. Call the `Human` class - `Human()`; this internally calls the `__call__` method of the type class (i.e., `type.__call__(Human, "Kian", "Pirfalak")`).
2. `type.__call__` will first call the `__new__` method defined on the `Human` class. If the `__new__` method is not defined on the `Human` class, the `__new__` method of the object class will be called.
3. The `__new__` method will the return the object of type `Human` i.e. `human_obj`
4. Now, `type.__call__` will call the `__init__` method defined on the `Human` class with `human_obj` as the first argument. This `human_obj` will be `self` in the `__init__` method.
5. The `__init__` method will initialize the `human_obj` with the `first_name` as Kian and the `last_name` as Pirfalak. The `__init__` method will not return anything.
6. In the end, `type.__call__` will return the `human_obj` object.

As per the `type.__call__` definition, whenever we create a new object, the `__new__` method will always be called, but calling the `__init__` method depends on the output of the `__new__` method. The `__init__` method will be called only if the `__new__` method returns an object of type `Human` class or a subclass of the `Human` class.

## Conclusion

In this article, we explored the `__new__`, `__init__`, and `__call__` magic methods and discussed Metaclass in Python. In doing so, we have a better understanding of the object creation and initialization processes in Python.